# **LIMPIEZA ENSAMBLE Y ANÁLISIS DE GENOMAS DE VIRUS DE INFLUENZA**

## **PREPARAR AMBIENTE INICIAL**

Definir variables útiles, que corresponden al nombre de la muestra y a los nombres de los archivos fastq.gz

In [ ]:
MUESTRA=""
FASTQ1=""
FASTQ2=""

Cambiar el directorio a la carpeta a procesar. Ya debe contener los archivos FASTQ a procesar. <span style="font-size: 150%; color: red;"> **Modificar dirección.**</span>

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

Crear un directorio para depositar el análisis de calidad de lecturas crudas, y una carpeta para depositar las lecturas crudas. Mover ahí los archivos con las lecturas crudas:

In [ ]:
mkdir CALIDAD_CRUDA
mkdir RAW
mv CPA* ./RAW

## **LIMPIEZA DE ARCHIVOS FASTQ**

Ejectuar análisis de calidad inicial:

In [ ]:
conda activate calidad
fastqc ./RAW/${FASTQ1} ./RAW/${FASTQ2} --outdir ./CALIDAD_CRUDA --threads 20

Revisar los archivos HTML de salida para identificar parámetros que ayuden a la limpieza. Esto se puede lograr transfiriendo los archivos HTML a una computadora local y visualizando en un navedador de internet.

En caso de requerir limpiar los archivos crudos manualmente, se puede usar lo siguiente:

In [ ]:
# Descomentar y modificar para eliminar mosaicos completos:
# seqkit grep -v -r -p [Patron por ejemplo: ':2110:'] [Archivo_R1] | pigz --best -p26 >[Archivo_R1_filtrado]
# seqkit grep -v -r -p [Patron por ejemplo: ':2110:'] [Archivo_R2] | pigz --best -p26 >[Archivo_R2_filtrado]

# Descomentar y modificar para sobreescribir las variables con los archivos a usar:
# FASTQ1=""
# FASTQ2=""

Ejecutar el programa de limpieza. <span style="font-size: 150%; color: red;"> **Ajustar parámetros de acuerdo a las necesidades.**</span> Cuidado al modificar los cores (ver documentación de trim_galore).

In [ ]:
trim_galore --paired --retain_unpaired --gzip --fastqc --cores 3 --clip_R1 18 --clip_R2 18 --three_prime_clip_R1 4 --three_prime_clip_R2 16 --length 120 -o TRIMMING/ ./RAW/${FASTQ1} ./RAW/${FASTQ2}

Revisar los archivos HTML de salida para verificar los resultados de la limpieza. Queda a consideración de la experiencia del usuario el quedar conforme con la limpieza realizada.

Desactivar ambiente calidad

In [ ]:
conda deactivate calidad

## **PREPARAR LIGAS A ARCHIVOS A USAR**

Encontrar la direccion de los archivos generados por trim galore:

In [ ]:
FASTQ1_P=$(find TRIMMING/ -type f -name "*val_1.fq.gz")
FASTQ2_P=$(find TRIMMING/ -type f -name "*val_2.fq.gz")
FASTQ1_U=$(find TRIMMING/ -type f -name "*unpaired_1.fq.gz")
FASTQ2_U=$(find TRIMMING/ -type f -name "*unpaired_2.fq.gz")

Crear ligas simbolicas a los archivos a usar:

In [ ]:
ln -s ${FASTQ1_P} reads_r1_tr
ln -s ${FASTQ2_P} reads_r2_tr
ln -s ${FASTQ1_U} reads_u1_tr
ln -s ${FASTQ2_U} reads_u2_tr

## **ALINEAMIENTO DE LECTURAS CONTRA BASE DE DATOS DE INFLUENZA Y FILTRADO**

Crear directorio para los alineamientos:

In [ ]:
mkdir -p ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING

Activar ambiente conda alineamiento

In [ ]:
conda activate alineamiento

Hacer los alineamientos de las secuencias pareadas y no pareadas contra la base de datos viral preparada en la UASIP.

In [ ]:
#Alineamiento de secuencias pareadas 
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_r1_tr reads_r2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/paired.sam 

#Alineamiento de secuencias no pareadas de R1
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u1_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r1.sam

#Alineamiento de secuencias no pareadas de R2
bwa-mem2 mem -t 26 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r2.sam

Entrar al directorio donde se depositaron los resultados del alineamiento, formatear los archivos sam resultantes en un archivo bam ordenado y regresar al directorio del proyecto.

In [ ]:
cd ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING 
samtools merge -u - paired.sam unpaired_r1.sam unpaired_r2.sam | samtools sort -o all_sorted.bam && rm paired.sam unpaired_r1.sam unpaired_r2.sam
cd -

Crear ocho directorios para depositar las lecturas por el segmento al cual alinearon.

In [ ]:
mkdir ALINEAMIENTO/BWA/S{1..8}

Ejecutar el siguiente programa. Este programa abre el archivo bam, elimina alineamientos con alineamientos secundarios, y obtiene las lecturas por cada segmento. Se analiza, por segmento, el formato CIGAR asociado de cada lectura. Se filtran aquellas secuencias que alinearon en al menos 70% de sus bases. Se guardan los nombres de las lecturas pertinentes y se extraen las secuencias de los archivos fastq limpios usando dichos nombres. Los archivos producidos se guardan en las ubicaciones pertinentes.

In [ ]:
~/analisis_influenza/bin/separar_seqs_segmentos.sh

Desactivar ambiente alineamiento

In [ ]:
conda deactivate alineamiento

## **ENSAMBLE DE NOVO DE GENOMAS**

Activa una terminal tmux para que se encargue de enviar los procesos aunque salgamos del servidor

In [ ]:
tmux new -s ensamble

Pegar el siguiente código de instrucciones en la terminal tmux. <span style="font-size: 150%; color: red;"> **Ajustar parámetros de acuerdo a las necesidades.**</span> Cuidado de pegar con tamaño de letra pequeño, si no, temux cambia de orden algunas instrucciones. Este paso está diseñado para ejecutarse sobre todos los segmentos virales. Es importante revisar cada archivo log de cada subcarpeta para verificar que se haya ejecutado todos los procesos, desde el kmer 11 al kmer 127. Se hace en terminal tmux para que el ciclo no caiga cuando se termine la conexion con el servidor. En caso de que el servidor se caiga y evite que algunos segmentos terminen de ensamblarse, modificar el ciclo for y volver a correr desde el segmento que quedó incompleto.

In [ ]:
for i in {1..8}
do
    ID="S${i}"
    id="s${i}"
    LOG="./ENSAMBLE/SPADES/${ID}/log_11_127_${ID}"
    mkdir -p "./ENSAMBLE/SPADES/${ID}"

    TMP_SCRIPT=$(mktemp /tmp/run_${ID}_XXXXXX.sh)

    cat > "$TMP_SCRIPT" <<EOF
    #!/bin/bash

    echo "=== Iniciando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    ~/analisis_influenza/bin/ensamble.sh \\
        -a "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r1.fq.gz" \\
        -b "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r2.fq.gz" \\
        -x "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u1.fq.gz" \\
        -y "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u2.fq.gz" \\
        -t 28 --kini 11 --kstep 2 --kfin 127 -o ./ENSAMBLE/SPADES/${ID}/OUT_11_127 >> "$LOG" 2>&1

    echo "=== Finalizando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    rm -- "\$0"
EOF

    chmod +x "$TMP_SCRIPT"
    nohup "$TMP_SCRIPT" &
    wait
done

Se puede cerrar sesion con Ctrl + B, y luego presionar D. Para reconectar la terminal tmux usar:

In [ ]:
tmux attach -t ensamble

<span style="font-size: 150%; color: gold;"> Revisar el archivo scaffolds.fasta del kmer donde se obtuvo la secuencia más larga. Usar NCBI y Expasy. Se debe poder detectar un CDS completo.</span>

## **ENSAMBLE CON CONTIGS CONFIABLES**

En caso de que el ensamblaje no haya logrado abarcar todo el CDS (longitud menor que la esperada mínima), se puede optar por realizar un reensamblaje con el archivo que especifique un contig confiable el cual será la secuencia más larga que se pudo ensamblar. Se deben revisar los análisis que arrojan las instrucciones anteriores y en funcion de eso determinar cual y donde se creará el archivo de contigs confiable. Se puede usar la siguiente instruccion:

In [ ]:
# Descomentar y modificar de acuerdo a necesidades
# seqkit head -n1 ./ENSAMBLE/SPADES/[SEGMENTO]/OUT_11_127/[KMER]/scaffolds.fasta >./ENSAMBLE/SPADES/[SEGMENTO]/OUT_11_127/[KMER]/trusted.fasta

Una vez identificado creado el archivo de contig confiable, modificar el script siguiente de acuerdo a las necesidades.  
<span style="font-size: 150%; color: red;"> **Revisar con cuidado el segmento a ensamblar y la dirección del archivo trusted.**</span>

In [ ]:
for i in 1 # Especificar segmento a ensamblar
do
    ID="S${i}"
    id="s${i}"
    LOG="./ENSAMBLE/SPADES/TRUSTED/${ID}/log_11_127_${ID}"
    mkdir -p "./ENSAMBLE/SPADES/TRUSTED/${ID}"

    TMP_SCRIPT=$(mktemp /tmp/run_${ID}_XXXXXX.sh)

    cat > "$TMP_SCRIPT" <<EOF
    #!/bin/bash

    echo "=== Iniciando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    ~/analisis_influenza/bin/ensamble.sh \\
        -a "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r1.fq.gz" \\
        -b "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r2.fq.gz" \\
        -x "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u1.fq.gz" \\
        -y "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u2.fq.gz" \\
        --trusted ./ENSAMBLE/SPADES/${ID}/OUT_11_127/K127/trusted.fasta \\ # ¡¡¡REVISAR LA DIRECCIÓN!!!
        -t 28 --kini 11 --kstep 2 --kfin 127 -o ./ENSAMBLE/SPADES/TRUSTED/${ID}/OUT_11_127 >> "$LOG" 2>&1

    echo "=== Finalizando proceso para ${ID} el \$(date) ==="  >> "$LOG"

    rm -- "\$0"
EOF

    chmod +x "$TMP_SCRIPT"
    nohup "$TMP_SCRIPT" &
    wait
done

<hr style="border: none; height: 5px; background-color: gold">

Los procesos anteriores pueden continuar durante la noche, asi es que <span style="font-size: 150%; color: gold"> es importante recargar variables iniciales, la posicion de trabajo y activar el ambiente alineamiento: </span>

In [ ]:
conda activate alineamiento
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

## **PREPARACION DE ARCHIVO DE RESULTADOS DE ENSAMBLE Y BLAST, Y CALCULAR PROFUNDIDADES**

Hacer loop para unir todos los resultados de blast (si se hizo por partes) y analizar los resultados, extraer secuencias, revisar orientacion y calcular profundidad:

In [ ]:
# Columnas deben ser similares a los archivos producidos en ensamble_analyze_blast
echo -e "SEGMENT\
\tKMER\
\tNODE\
\tLENGTH\
\tSLEN\
\tPERCENTAGE\
\tCOV\
\tSUBJECT_STRAND\
\tBLAST_REFERENCE\
\tBLAST_DESCRIPTION" >./ENSAMBLE/SPADES/all_segm_best_result_blast.txt

for i in {1..8}; do
    ID="S${i}"

    cd "./ENSAMBLE/SPADES/${ID}"

    ~/analisis_influenza/bin/merge_blast_results.sh

    ~/analisis_influenza/bin/ensamble_analyze_blast.sh ${ID}_blastn-careful_merged.txt
    
    sed -n '2p' best_result_${ID}_blastn-careful_merged.txt >>../all_segm_best_result_blast.txt

    cd -
done


Ingresar al directorio SPADES para los siguientes analisis:

In [ ]:
cd ./ENSAMBLE/SPADES

Obtener porcentajes de lecturas usadas por cada segmento:

In [ ]:
~/analisis_influenza/bin/unir_profundidades.sh \
    --S1 S1/PROFUNDIDAD/S1_profundidad \
    --S2 S2/PROFUNDIDAD/S2_profundidad \
    --S3 S3/PROFUNDIDAD/S3_profundidad \
    --S4 S4/PROFUNDIDAD/S4_profundidad \
    --S5 S5/PROFUNDIDAD/S5_profundidad \
    --S6 S6/PROFUNDIDAD/S6_profundidad \
    --S7 S7/PROFUNDIDAD/S7_profundidad \
    --S8 S8/PROFUNDIDAD/S8_profundidad

mv prof.tsv ${MUESTRA}_prof.tsv

~/analisis_influenza/bin/ensamble_cal_porc_lect_mapeadas.sh

Salir del ambiente alineamiento:

In [ ]:
conda deactivate alineamiento

## **GRAFICAR PROFUNDIDADES Y USO DE SECUENCIAS POR SEGMENTO**

In [ ]:
conda activate R
~/analisis_influenza/bin/graficar_profundidad.R --input_file ${MUESTRA}_prof.tsv --muestra ${MUESTRA}
~/analisis_influenza/bin/graficar_lecturas.R --input_file uso_de_lecturas.tsv --muestra ${MUESTRA}
conda deactivate

## **ANALISIS DE SUPTIFICACIÓN**

Hacer el análisis de subtipificación, donde se cuenten los subtipos de cada subject alineado por BLAST, y se obtenga una tabla con los mejores 5 subtipos.

In [ ]:
for i in {1..8}
do
    # Contruir el nombre del directorio
    DIR_S="S${i}"

    #Acceder al directorio
    cd ./${DIR_S}
    
    # Hacer análisis de subtipificacion. Consiste en obtener todos los HxNx de todos los hits de blast y determinar el subtipo más abundante.
    echo "Ejecutando análisis de subtipificación para ${DIR_S}"
    ~/analisis_influenza/bin/analisis_subtipificacion.sh best_result_${DIR_S}_sequence_corrected.fna
    echo " "

    # Regresar al directorio anterior
    cd -

    # Guardar resultados de cada segmento en un archivo común.
    (
        echo "Análisis de subtipificacion para ${DIR_S}"
        cat ./${DIR_S}/${DIR_S}_subtipos_top5.tsv
        echo ""
    ) >> "analisis_subtipificacion.tsv"
done

## **ANALISIS DE PATOGENICIDAD**

Este análisis sólo se realiza sobre el S4. El script buscará 4 o más residuos positivos entre el motivo PXnGLF (donde Xn son residuos positivos (K, H, R), principalmente).  
Se realiza sobre todos los marcos de lectura pero debe aparecer en los primeros tres puesto que se trabaja con las secuencias con orientacion corregida.

In [ ]:
cd S4
~/analisis_influenza/bin/analisis_patogenicidad.sh best_result_S4_sequence_corrected.fna >analisis_patogenicidad_S4.txt
cd -

## **PREPARAR FASTA FINAL**

Generar el fasta final. Se incluyen datos para incluir en la construcción de los headers de las secuencias. Son obligatorios y en caso de no contar con alguno de ellos dejar como NA, para garantizar consistencia en los encabezados.

In [ ]:
~/analisis_influenza/bin/renombrar_fastas.sh --host NA --origin NA --year 2025

## **PREDECIR ORFS**

Se ejecuta el siguiente script que corre el programa getorfs de EMBOSS. El objetivo es delimitar la secuencia del fasta que incluye el ORF de cada segmento viral (El más grande para ORFs solapantes). Posteriormente identifica la posicion de inicio y termino del ORF, obtiene la posición 15 nucleótidos antes del codon de inicio y 15 nucleotidos despues del codon de termino del ORF, y genera una tabla con toda esa información 

Se renombra el fasta cortado con el nombre de la muestra.

In [ ]:
~/analisis_influenza/bin/predecir_orf.sh --s1_max 2436 --s2_max 2471 --s3_max 2287 --s4_min 1700 --s4_max 1720 --s5_max 1646 --s6_max 1594 --s7_max 1226 --s8_max 1023 --input ${MUESTRA}.fna 

<span style="font-size: 150%; color: gold;"> Revisar a tabla manualmente. Asegurarse que el CDS esté correcto.</span> <span style="font-size: 150%; color: red;"> Corregir final para S7 y S8, porque hay splicing alternativo que no sobrelapa perfectamente.</span>

Luego, producir el fasta cortado y renombrarlo usando el nombre de la muestra.

In [ ]:
~/analisis_influenza/bin/cortar_fasta.sh tbl_longest_orf_range.tsv
mv fasta_cortado.fna ${MUESTRA}_cortado.fna

## **CORTAR PROFUNDIDADES**

Se toma el archivo de profundidades y la tabla y, en función de los inicios y terminos de CDS de cada secuencia, se recortan los bordes del archivo de profundidades en la columna correspondiente.

In [ ]:
conda run -n R Rscript ~/analisis_influenza/bin/cortar_profundidades.R --profundidades ${MUESTRA}_prof.tsv --rangos tbl_longest_orf_range.tsv --n_inicio 15 --n_final 15
mv profundidades_cortadas.tsv ${MUESTRA}_prof_cortadas.tsv

## **LIMPIAR AREA DE TRABAJO**

La siguiente instrucción elimina los archivos de grafos, las rutas que siguen los archivos de grafos asociadas a las secuencias, y archivos intermedios. Ejecutar en la carpeta de inicio de la muestra.

In [ ]:
find . \( -name "assembly_graph*" -o -name "*paths" -o -name "before_rr*" -o -name "mis*" -o -name "corrected" -o -name "pipeline_state" -o -name "split_input" -o -name "tmp" \) -exec rm -rf {} +